Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, divide en entrenamiento y prueba (20%), después divide los datos de entrenamiento tomando las primeras 1000 muestras para entrenamiento y el resto para validación).

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_digits

In [2]:
mnist = load_digits()
X = mnist.data
y = mnist.target

In [3]:
X.shape

(1797, 64)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_value = X_train[1000:]
y_value = y_train[1000:]
X_train = X_train[:1000]
y_train = y_train[:1000]

Entrena clasificadores individuales, que por lo menos alcancen 0.5 de desempeño en los datos de validación. Por ejemplo un clasificador DecisionTree, un clasificador SVM y un MLP.

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_clf = SVC(C=3, gamma=0.012)

svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_value)

# Calcula la tasa de aciertos de las predicciones del clasificador
tasa_aciertos = accuracy_score(y_value, y_pred)

print(f'Tasa de aciertos: {tasa_aciertos:.2f}')


Tasa de aciertos: 0.65


In [6]:
tree_clf = DecisionTreeClassifier(random_state=2, max_depth=5)
tree_clf.fit(X_train, y_train)

print(f'Tasa de aciertos: {tree_clf.score(X_test, y_test)}')

Tasa de aciertos: 0.6861111111111111


Entrena un modelo de Bagging y un modelo de Boosting para cada tipo de clasificador y compara los resultados usando los datos de validación.

In [19]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_depth=5),
                            n_estimators=50,
                            max_samples=100,
                            max_features = 1.0,
                            bootstrap = True,
                            oob_score=True,
                            n_jobs=-1,
                            random_state=2)
bag_clf.fit(X_train, y_train)
print(bag_clf.oob_score_)
print(bag_clf.score(X_value, y_value))

0.894
0.8924485125858124


In [20]:
bag_clf = BaggingClassifier(SVC(C=3, gamma=0.012),
                            n_estimators=50,
                            max_samples=100,
                            max_features = 1.0,
                            bootstrap = True,
                            oob_score=True,
                            n_jobs=-1,
                            random_state=2)
bag_clf.fit(X_train, y_train)
print(bag_clf.oob_score_)
print(bag_clf.score(X_test, y_test))

0.227
0.22777777777777777


In [16]:
from sklearn.ensemble import AdaBoostClassifier

svc = SVC(
    C=3, probability=True, gamma=.012
)

ada_clf = AdaBoostClassifier( svc,
                              n_estimators=3,
                              learning_rate=0.9,
                              random_state=0
                             )

ada_clf.fit(X_train, y_train)

svc.fit(X_train, y_train)

print("SVC score:")
print(svc.score(X_test, y_test))
print("SVC Adaboost score:")
print(ada_clf.score(X_test, y_test))

c:\Users\anaso\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


SVC score:
0.6972222222222222
SVC Adaboost score:
0.7388888888888889


In [21]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier( DecisionTreeClassifier(max_depth=5),
                              n_estimators=3,
                              learning_rate=0.9,
                              random_state=0
                             )

ada_clf.fit(X_train, y_train)

svc.fit(X_train, y_train)

print("Decision Tree Adaboost:")
print(ada_clf.score(X_test, y_test))

c:\Users\anaso\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Decision Tree Adaboost:
0.7222222222222222


Al ensamble con mejor resultado evalualo con los datos de prueba